In [0]:
import en_core_web_sm
nlp = en_core_web_sm.load()
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import pandas as pd
import numpy as np
from time import time 
import multiprocessing
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

#file_name = 'document_classification\Resume1.txt'
#mat=[][2]
import os


main=[]
#print(files)
for i in os.listdir("/content/drive/My Drive/resume/Training & Test/Resume"):
     if i != ".ipynb_checkpoints":
      x=i
      try:
        fname = open("/content/drive/My Drive/resume/Training & Test/Resume/"+x).read()
      except UnicodeDecodeError:
         continue
        
      f1=re.sub(r'\W+', ' ',fname)
    
      wordnet_lemmatizer = WordNetLemmatizer()
    
      words = nltk.word_tokenize(f1)
    
      f2=" "
      for w in words:
          f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
    
      doc = nlp(f2)
    
      tokens = [token.text for token in doc if not token.is_stop]
    
      l=[]
      l.append(tokens)
      l.append("1")
      main.append(l)

for i in os.listdir("/content/drive/My Drive/resume/Training & Test/Non Resume"):
     if i != ".ipynb_checkpoints":
       x=i
       try:
          fname = open("/content/drive/My Drive/resume/Training & Test/Non Resume/"+x).read()
       except UnicodeDecodeError:
          continue
        
       f1=re.sub(r'\W+', ' ',fname)
       wordnet_lemmatizer = WordNetLemmatizer()
       words = nltk.word_tokenize(f1)
       f2=" "
       for w in words:
           f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
       doc = nlp(f2)
    
       tokens = [token.text for token in doc if not token.is_stop]
      
       l=[]
       l.append(tokens)
       l.append("0")
       main.append(l)



df = pd.DataFrame(main, columns = ['Tokens', 'Label'])







LookupError: ignored

In [0]:

pred = []
for i in os.listdir("/content/drive/My Drive/resume/Prediction/Resume"):
    if i != ".ipynb_checkpoints":
      x=i
      try:
        fname = open("/content/drive/My Drive/resume/Prediction/Resume/"+x).read()
      except UnicodeDecodeError:
        continue
        
      f1=re.sub(r'\W+', ' ',fname)
      wordnet_lemmatizer = WordNetLemmatizer()
      words = nltk.word_tokenize(f1)
      f2=" "
      for w in words:
          f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
      doc = nlp(f2)
   
      tokens = [token.text for token in doc if not token.is_stop]
    
      l=[]
      l.append(tokens)
      pred.append(l)

for i in os.listdir("/content/drive/My Drive/resume/Prediction/Non Resume"):
    if i != ".ipynb_checkpoints":
      x=i
      try:
          fname = open("/content/drive/My Drive/resume/Prediction/Non Resume/"+x).read()
      except UnicodeDecodeError:
          continue
          
      f1=re.sub(r'\W+', ' ',fname)
      wordnet_lemmatizer = WordNetLemmatizer()
      words = nltk.word_tokenize(f1)
      f2=" "
      for w in words:
          f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
      doc = nlp(f2)
    
      tokens = [token.text for token in doc if not token.is_stop]
      
      l=[]
      l.append(tokens)
      pred.append(l)


df_pred = pd.DataFrame(pred, columns=['Tokens'])
df_pred

,Tokens
0,"[ , Computer, Science, Faculty, New, Delhi, D..."
1,"[ , Isabelle, Smith, rn, cphon, Sometown, TX,..."
2,"[ , BIODATA, G, Pranathi, 8143958515, 16wh1a0..."
3,"[ , PAYMENT, RECEIPT, Guest, Sanjay, Date, 31..."
4,"[ , Cody, Fredrickson, 123, 456, 7891, cfredr..."
5,"[ , Jax, Sampson, 111, 789, 3456, jax, sampso..."


In [0]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
cores = multiprocessing.cpu_count() 
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

t = time()

w2v_model.build_vocab(df["Tokens"], progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

print(w2v_model.corpus_count)
w2v_model.train(df["Tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

NameError: ignored

In [0]:
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix

y = df['Label'].values
X = np.array(df["Tokens"])

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)

matrix = vectorizer.fit_transform([x for x in X_train])

tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    
    count = 0.
    for word in tokens:
        try:
    
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    
    
    return vec

train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_train)])


test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_test)])
train_vecs_w2v = scale(train_vecs_w2v)


test_vecs_w2v = scale(test_vecs_w2v)




vocab size : 641


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:


X_pred = np.array(df_pred["Tokens"])


pred_vec_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_pred)])
pred_vec_w2v = scale(pred_vec_w2v)


print ('shape for training set : ',train_vecs_w2v.shape,
      '\nshape for test set : ', test_vecs_w2v.shape,
       '\nshape for predicted set : ', pred_vec_w2v.shape,) 


shape for training set :  (135, 300) 
shape for test set :  (59, 300) 
shape for predicted set :  (6, 300)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate

model1 = Sequential()
 
model1.add(Dense(128, activation='relu', input_dim=300))
model1.add(Dropout(0.7))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 128)               38528     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [0]:
import matplotlib.pyplot as plt

history = model1.fit(train_vecs_w2v, y_train, epochs=40, batch_size=50,
                   validation_data=(test_vecs_w2v,y_test))
loss, accuracy = model1.evaluate(train_vecs_w2v, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))


loss, accuracy = model1.evaluate(test_vecs_w2v, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))



Train on 135 samples, validate on 59 samples
Epoch 1/40
135/135 [==============================] - 0s 3ms/step - loss: 1.1296 - acc: 0.5556 - val_loss: 0.4292 - val_acc: 0.7288
Epoch 2/40
135/135 [==============================] - 0s 88us/step - loss: 0.4618 - acc: 0.7778 - val_loss: 0.2755 - val_acc: 0.8644
Epoch 3/40
135/135 [==============================] - 0s 80us/step - loss: 0.3272 - acc: 0.8444 - val_loss: 0.2178 - val_acc: 0.8983
Epoch 4/40
135/135 [==============================] - 0s 101us/step - loss: 0.2571 - acc: 0.8667 - val_loss: 0.1775 - val_acc: 0.9153
Epoch 5/40
135/135 [==============================] - 0s 89us/step - loss: 0.1878 - acc: 0.9259 - val_loss: 0.1538 - val_acc: 0.9322
Epoch 6/40
135/135 [==============================] - 0s 90us/step - loss: 0.2023 - acc: 0.9037 - val_loss: 0.1407 - val_acc: 0.9492
Epoch 7/40
135/135 [==============================] - 0s 96us/step - loss: 0.1837 - acc: 0.9407 - val_loss: 0.1318 - val_acc: 0.9492
Epoch 8/40
135/135 [====

In [0]:

l = model1.predict(pred_vec_w2v, verbose=False)
print(l)

[[9.9992025e-01]
 [9.9994910e-01]
 [1.0941011e-02]
 [2.7500442e-01]
 [7.1451475e-04]
 [8.6270487e-03]]


In [0]:

! ls sample_data/

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md
